In [1]:
pip install transformers datasets torch scikit-learn numpy pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
import pandas as pd

# Load dataset
df = pd.read_csv("goemotions.csv")

# Select relevant columns: 'text' + chosen emotions
TEXT_COLUMN = "text"
SELECTED_LABELS = ["gratitude", "approval", "anger", "joy", "sadness"]

df = df[[TEXT_COLUMN] + SELECTED_LABELS]  # Keep only required columns
df.dropna(inplace=True)  # Remove missing values

print("\nDataset Preview:\n", df.head())


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Convert text and labels to lists
text_data = df[TEXT_COLUMN].tolist()
labels = df[SELECTED_LABELS].values  # Convert selected labels to NumPy array

# Split into train and test sets (80-20)
X_train, X_test, y_train, y_test = train_test_split(text_data, labels, test_size=0.2, random_state=42)

# Convert text to TF-IDF features (faster than BERT)
vectorizer = TfidfVectorizer(max_features=5000, stop_words="english")  # Limit features for speed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("\nTF-IDF Shape:", X_train_tfidf.shape)


In [8]:
from torch.utils.data import Dataset

class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx],
        }

# Create PyTorch datasets
train_dataset = EmotionDataset(train_encodings, train_labels)
test_dataset = EmotionDataset(test_encodings, test_labels)

In [9]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load model for multi-label classification
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(SELECTED_LABELS),  # Match selected labels
    problem_type="multi_label_classification"
)

# Training arguments optimized for CPU
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,  # Reduce batch size for CPU
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=2,  # Limit saved checkpoints to save space
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train model
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Moon-Study\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 